In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner

In [2]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [3]:
build_model(keras_tuner.HyperParameters())

In [4]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="C:/Users/seeho/data/tuner",
    project_name="helloworld",
)

In [5]:
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 00m 23s]
val_accuracy: 0.9604000151157379

Best val_accuracy So Far: 0.9626999795436859
Total elapsed time: 00h 01m 15s
INFO:tensorflow:Oracle triggered exit


In [7]:
"""When search is over, you can retrieve the best model(s). The model is saved at its best performing epoch
evaluated on the validation_data."""
# Get the top revision by specifying 'num_models'
models = tuner.get_best_models(num_models=1)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 224)               175840    
                                                                 
 dense_1 (Dense)             (None, 32)                7200      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 183,370
Trainable params: 183,370
Non-trainable params: 0
_________________________________________________________________


In [8]:
"""summary what you will be getting"""
tuner.results_summary()

Results summary
Results in C:/Users/seeho/data/tuner\helloworld
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
units_0: 224
activation: tanh
dropout: True
lr: 0.000966963677831812
units_1: 32
Score: 0.9626999795436859
Trial summary
Hyperparameters:
num_layers: 2
units_0: 128
activation: relu
dropout: False
lr: 0.007125241036137976
units_1: 192
Score: 0.9604000151157379
Trial summary
Hyperparameters:
num_layers: 1
units_0: 320
activation: tanh
dropout: True
lr: 0.0004900707206926026
units_1: 224
Score: 0.9527499973773956


In [9]:
"""retrieve the best hyperparameters. Retrain the final model for the last time"""
# Get the top 5 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3499 - accuracy: 0.9004
Epoch 2/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1668 - accuracy: 0.9525
Epoch 3/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1148 - accuracy: 0.9662


In [10]:
"""summary what you will be getting"""
tuner.results_summary()

Results summary
Results in C:/Users/seeho/data/tuner\helloworld
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
units_0: 224
activation: tanh
dropout: True
lr: 0.000966963677831812
units_1: 32
Score: 0.9626999795436859
Trial summary
Hyperparameters:
num_layers: 2
units_0: 128
activation: relu
dropout: False
lr: 0.007125241036137976
units_1: 192
Score: 0.9604000151157379
Trial summary
Hyperparameters:
num_layers: 1
units_0: 320
activation: tanh
dropout: True
lr: 0.0004900707206926026
units_1: 224
Score: 0.9527499973773956
